In [2]:
import sys
import torch
sys.path.append('../')
import cace

### Model load

In [27]:
cace_nnp = torch.load('./CsPbBr_3_m0_lr.pth', map_location='cpu')
cace_representation = cace_nnp.models[1].representation
q_module=cace_nnp.models[1].output_modules[0]
cutoff=cace_representation.cutoff

/var/folders/r_/08n4lh1d2_129d09d1b_scv80000gn/T/ipykernel_43347/836237146.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cace_nnp = torch.load('./CsPbBr_3_m0_lr.pth', 

### data preparation

In [41]:
from cace.tools import torch_geometric
from cace.data import AtomicData
from ase.io import read

def data_ready(atoms):
    data = torch_geometric.dataloader.DataLoader(
        dataset=[
                AtomicData.from_atoms(
                atoms, cutoff=cutoff
                )
        ],
        batch_size=1,
        shuffle=False,
        drop_last=False,)
    batch = next(iter(data))
    dict = batch.to_dict()
    return dict

In [69]:
data = './CsPbBr_3.xyz' 

#cubic cell
cubic = read(data, index=0)
print('check cubic cell')
print(cubic.get_cell_lengths_and_angles())
cubic_dict = data_ready(cubic)
cubic_rep = cace_representation(cubic_dict)
cubic_q = q_module(cubic_rep)

#triclininc cell
tri = read(data, index=-1)
print('check triclinic cell')
print(tri.get_cell_lengths_and_angles())
tri_dict = data_ready(tri)
tri_rep = cace_representation(tri_dict)
tri_q = q_module(tri_rep)

#cubic-triclinic supercell
from pymatgen.io.ase import AseAtomsAdaptor
adaptor = AseAtomsAdaptor()
s_cubic = adaptor.get_structure(cubic)
s_tri = s_cubic.make_supercell([[1, 1, 1] , [0, 1, 0] , [0, 0, 1]])
tri_cubic = AseAtomsAdaptor.get_atoms(s_tri)
print('check cubic-triclinic supercell')
print(tri_cubic.get_cell_lengths_and_angles())

tri_c_dict = data_ready(tri_cubic)
tri_c_rep = cace_representation(tri_c_dict)
tric_q = q_module(tri_c_rep)

check cubic cell
[11.86 11.86 11.86 90.   90.   90.  ]
check triclinic cell
[11.86903086 11.88502143 11.79185929 92.1600353  87.6963706  92.42096124]
check cubic-triclinic supercell
[20.54212258 11.86       11.86       90.         54.73561032 54.73561032]


In [70]:
ep = cace.modules.EwaldPotential(dl=2,
                    sigma=1.0,
                    
                    feature_key='q',
                    output_key='ewald_potential',
                    remove_self_interaction=False,
                    compute_field=True,
                   aggregation_mode='sum')

### single cell results

In [37]:
result_cubic = ep(cubic_q)['ewald_potential']
result_cubic_pot = ep(cubic_q)['q_field']

In [38]:
print(result_cubic)

tensor([0.4710], grad_fn=<SumBackward1>)


In [39]:
result_tri = ep(tri_q)['ewald_potential']

In [40]:
print(result_tri)

tensor([0.4179], grad_fn=<SumBackward1>)


In [64]:
result_tri_cub = ep(tric_q)['ewald_potential']
result_tri_cub_pot = ep(tric_q)['q_field']

In [65]:
print(result_tri_cub)

tensor([0.4710], grad_fn=<SumBackward1>)


In [66]:
print(torch.allclose(result_cubic, result_tri_cub, atol=1e-2))
print(torch.allclose(result_cubic_pot, result_tri_cub_pot, atol=1e-5))

True
True


### Extensive energy test

In [ ]:
#cubic cell
cubic_double = cubic.repeat((2, 2, 2))
print(cubic_double.get_cell_lengths_and_angles())
cubic_double_dict = data_ready(cubic_double)
cubic_double_rep = cace_representation(cubic_double_dict)
cubic_double_q = q_module(cubic_double_rep)

result_cubic_2 = ep(cubic_double_q)['ewald_potential']

print(result_cubic)
print(result_cubic_2/result_cubic)

[23.72 23.72 23.72 90.   90.   90.  ]
tensor([0.4710], grad_fn=<SumBackward1>)
tensor([8.0000], grad_fn=<DivBackward0>)


In [59]:
#triclininc cell
tri_double = tri.repeat((2, 2, 2))
print(tri_double.get_cell_lengths_and_angles())
tri_double_dict = data_ready(tri_double)
tri_double_rep = cace_representation(tri_double_dict)
tri_double_q = q_module(tri_double_rep)

result_tri_2 = ep(tri_double_q)['ewald_potential']

print(result_tri)
print(result_tri_2/result_tri)

[23.73806172 23.77004285 23.58371858 92.1600353  87.6963706  92.42096124]
tensor([0.4179], grad_fn=<SumBackward1>)
tensor([8.0000], grad_fn=<DivBackward0>)


/opt/miniconda3/envs/torch/lib/python3.10/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


In [67]:
#cubic-triclinic supercell
tri_cubic_double = tri_cubic.repeat((2, 2, 2))
print(tri_cubic_double.get_cell_lengths_and_angles())
tri_cubic_double_dict = data_ready(tri_cubic_double)
tri_cubic_double_rep = cace_representation(tri_cubic_double_dict)
tri_cubic_double_q = q_module(tri_cubic_double_rep)

result_tri_cub_2 = ep(tri_cubic_double_q)['ewald_potential']

print(result_tri_cub)
print(result_tri_cub_2/result_tri_cub)

[41.08424516 23.72       23.72       90.         54.73561032 54.73561032]
tensor([0.4710], grad_fn=<SumBackward1>)
tensor([8.0000], grad_fn=<DivBackward0>)
